In [ ]:

!pip install numpy
!pip install torch
!pip install matplotlib
!pip install scikit-learn

In [ ]:
import os
import random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np


class EmotionAwareDataset(torch.utils.data.Dataset):
    def __init__(self, base_dataset,
                 disgust_transform,
                 normal_transform):
        self.base_dataset = base_dataset
        self.base_dataset.transform = None 
        
        self.disgust_transform = disgust_transform
        self.normal_transform = normal_transform
        
        self.disgust_idx = self.base_dataset.class_to_idx['disgust']
        
    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        image, label = self.base_dataset[idx]
        
        if label == self.disgust_idx:
            image = self.disgust_transform(image)
        else:
            image = self.normal_transform(image)
        return image, label
class DataModel:
    def __init__(self, data_dir, batch_size, img_size=96):
        self.data_dir = data_dir 
        self.batch_size = batch_size
        self.img_size = img_size
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

    def get_transforms(self):
        disgust_transform = transforms.Compose([
            transforms.RandomResizedCrop(self.img_size, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(0.2, 0.2, 0.1),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])
        normal_transform = transforms.Compose([
            transforms.RandomResizedCrop(self.img_size, scale=(0.9, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2), # Giúp model không quá phụ thuộc vào màu sắc
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std),
            transforms.RandomErasing(p=0.25, scale=(0.02, 0.15), ratio=(0.3, 3.3))
        ])
        test_transform = transforms.Compose([
            transforms.Resize((self.img_size, self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

        return {
            'disgust': disgust_transform,
            'normal': normal_transform,
            'test': test_transform
        }

    def create_data(self): 
        ts = self.get_transforms()
        
        train_path = os.path.join(self.data_dir, 'train')
        test_path = os.path.join(self.data_dir, 'test')

        base_train = datasets.ImageFolder(root=train_path)
        train_set = EmotionAwareDataset(
            base_train,
            disgust_transform=ts['disgust'], 
            normal_transform=ts['normal']
        )
        
        test_set = datasets.ImageFolder(root=test_path, transform=ts['test'])
        
        train_loader = DataLoader(train_set, batch_size=self.batch_size,
                                  shuffle = True,
                                  num_workers=4, pin_memory=True)
        test_loader = DataLoader(test_set, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)

        return train_loader, test_loader, len(test_set.classes), test_set.classes
dm = DataModel(data_dir='/kaggle/input/datasets/anmeo6/datasett/dataset', batch_size=32)
train_loader, test_loader, num_classes, names = dm.create_data()

In [ ]:
import torch
import torch.nn as nn
def conv3x3(in_channels,out_channels,stride=1):
    return nn.Conv2d(in_channels,out_channels,kernel_size= 3,stride=stride,padding= 1, bias= False)
class Block(nn.Module):
    expansion = 1
    def __init__(self, inchannels,outchannels,stride=1,downsample = None):
        super(Block,self).__init__()
        self.conv1 = conv3x3(inchannels,outchannels,stride)
        self.bn1 = nn.BatchNorm2d(outchannels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(outchannels,outchannels)
        self.bn2 = nn.BatchNorm2d(outchannels)
         
        self.downsample = downsample
        self.stride =stride

    def forward(self,x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out
class Resnet(nn.Module):
    def __init__(self,block,layer,num_classes):
        super(Resnet,self).__init__()
        self.inchannels = 64
        self.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=(1,1),bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self.layer(block, 64,layer[0])
        self.layer2 = self.layer(block, 128,layer[1],stride = 2)
        self.layer3 = self.layer(block, 256,layer[2],stride = 2)
        self.layer4 = self.layer(block, 512,layer[3],stride = 2)
        
        self.avgpooling = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Sequential(
            nn.Linear(512 * block.expansion, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(p=0.4), 
            nn.Linear(512, num_classes)
        )
    def layer(self,block,channels,blocks,stride = 1):
        downsample = None
        if stride != 1 or self.inchannels != channels*block.expansion:
            downsample = nn.Sequential(nn.Conv2d(self.inchannels,channels*block.expansion, kernel_size= 1,stride = stride, bias = False)
                                       ,nn.BatchNorm2d(channels*block.expansion))
        layers = []
        layers.append(block(self.inchannels,channels,stride,downsample))
        self.inchannels = channels*block.expansion
        for i in range(1,blocks):
            layers.append(block(self.inchannels,channels))
        return nn.Sequential(*layers)
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpooling(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x
def Resnet18(num_classes):
        return Resnet(Block,[2,2,2,2],num_classes)
model = Resnet18(num_classes=num_classes)

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing = 0.1)
optimizer = optim.AdamW([
    {'params': model.conv1.parameters(), 'lr': 1e-5},
    {'params': model.layer1.parameters(), 'lr': 1e-5},
    {'params': model.layer2.parameters(), 'lr': 5e-5},
    {'params': model.layer3.parameters(), 'lr': 1e-4},
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(), 'lr': 5e-4},
], weight_decay=5e-2)
epochs = 50
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
max1 = 0
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for i, (inputs,labels) in enumerate(train_loader):
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        preds = torch.argmax(outputs, 1)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_acc += torch.sum(preds == labels.data)
    epoch_train_loss = train_loss / len(train_loader.dataset)
    epoch_train_acc = train_acc.double() / len(train_loader.dataset)
    scheduler.step()
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    with torch.no_grad(): 
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            test_acc += torch.sum(preds == labels.data)
    epoch_test_loss = test_loss / len(test_loader.dataset)
    epoch_test_acc = test_acc.double() / len(test_loader.dataset)
    if epoch_test_acc > max1:
        max1 = epoch_test_acc
        torch.save(model, 'phanloaicamxuc_7.pth')
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train - Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f}")
    print(f"test   - Loss: {epoch_test_loss:.4f} Acc: {epoch_test_acc:.4f}")
model = torch.load('phanloaicamxuc_7.pth',weights_only=False) 
model.to(device)

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report # Thêm classification_report

def plot_confusion_matrix(model, test_loader, device, classes):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    
    # 1. Tính toán và In báo cáo chi tiết (Precision, Recall, F1)
    print("\n" + "="*20 + " CHI TIẾT ĐÁNH GIÁ (CLASSIFICATION REPORT) " + "="*20)
    report = classification_report(y_true, y_pred, target_names=classes)
    print(report)
    print("="*80)

    # 2. Tính ma trận nhầm lẫn
    cm = confusion_matrix(y_true, y_pred)
    
    # 3. Vẽ biểu đồ Heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Dự đoán (Predicted)')
    plt.ylabel('Thực tế (Actual)')
    plt.title('Confusion Matrix - Phân loại cảm xúc')
    plt.show()

# Gọi hàm sau khi train xong
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
plot_confusion_matrix(model, test_loader, device, classes)